## PostgreSQL Database Utility Functions

Simple utility functions for the PostgreSQL option strategies database.

In [28]:
import sys
import os
import pandas as pd
import psycopg2
import warnings
import json

# Suppress pandas warnings
warnings.filterwarnings('ignore', message='pandas only supports SQLAlchemy connectable')

# Setup paths
notebook_dir = os.getcwd()
project_root = os.path.dirname(notebook_dir) if 'database' in notebook_dir else notebook_dir
config_path = os.path.join(project_root, 'config')
database_path = os.path.join(project_root, 'database')

# Add to Python path
sys.path.insert(0, config_path)
sys.path.insert(0, database_path)

# Load PostgreSQL credentials from JSON file
with open(os.path.join(config_path, 'credentials.json'), 'r') as f:
    creds = json.load(f)

pg_creds = creds['database']['postgresql']

# Set environment variables BEFORE importing database_config
os.environ.update({
    'DB_TYPE': 'postgresql',
    'DB_HOST': pg_creds['host'],
    'DB_PORT': str(pg_creds['port']),
    'DB_NAME': pg_creds['database'],
    'DB_USER': pg_creds['user'],
    'DB_PASSWORD': pg_creds['password']
})

from database_config import DatabaseConfig, DatabaseConnection

# Force create PostgreSQL connection
config = DatabaseConfig()
config.db_type = 'postgresql'
config.pg_config = {
    'host': pg_creds['host'],
    'port': pg_creds['port'],
    'database': pg_creds['database'],
    'user': pg_creds['user'],
    'password': pg_creds['password']
}

db = DatabaseConnection(config)
print(f"✅ Connected to PostgreSQL: {db.config.pg_config['host']}")
print(f"📊 Database type: {db.config.db_type}")

def get_recent_data(limit=20):
    """Get recent records with valid tickers - newest dates at bottom"""
    query = "SELECT * FROM option_strategies WHERE ticker IS NOT NULL AND ticker != 'None' ORDER BY scrape_date DESC, id ASC LIMIT %s"
    return db.execute_query_df(query, (limit,))

# Fetch and display data
recent_data = get_recent_data(16)
print(f"📋 Showing last {len(recent_data)} records with all columns (newest dates at bottom):")

# Configure pandas display
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

print("="*100)
display(recent_data)
print("="*100)
print(f"✅ Displayed {len(recent_data)} records with {len(recent_data.columns)} columns")
print(f"📅 Date range: {recent_data['scrape_date'].min()} to {recent_data['scrape_date'].max()}")

✅ Connected to PostgreSQL: 35.204.11.121
📊 Database type: postgresql
📋 Showing last 16 records with all columns (newest dates at bottom):


,id,scrape_date,strategy_type,tab_name,ticker,er,trigger_price,strike_price,strike_buy,strike_sell,estimated_premium,last_price_when_checked,timestamp_of_price_when_last_checked,item_id,options_expiry_date,date_info,timestamp_of_trigger,strategy_status,price_when_triggered,price_when_order_placed,premium_at_order,premium_when_last_checked,timestamp_of_order
0,551,2025-08-20T04:41:19.203629,Bull Put,Minimal Risk 97-99% accuracy > longer expiry,TSM,0,226.09,sell 215.0 - buy 205.0,205.0,215.0,143.0,228.62,2025-08-20T20:38:08.799913,2,2025-09-12,August,2025-08-20T13:51:13.415790,order placed,225.93,None,None,143.0,2025-08-20T20:37:57.328838
1,550,2025-08-20T04:41:19.107808,Bull Put,Minimal Risk 97-99% accuracy > longer expiry,MSFT,0,496.85,sell 470.0 - buy 460.0,460.0,470.0,120.0,505.60,2025-08-20T20:38:08.773127,1,2025-09-12,August,None,None,NaN,None,None,NaN,None
2,549,2025-08-20T04:41:11.956848,Bull Put,Mild Risk 95-97% accuracy > longer expiry,TQQQ,0,84.55,sell 80.0 - buy 70.0,70.0,80.0,178.0,87.57,2025-08-20T20:38:08.649036,4,2025-09-12,August,2025-08-20T14:52:32.959323,order placed,84.42,None,None,178.0,2025-08-20T20:38:01.700322
3,548,2025-08-20T04:41:11.869861,Bull Put,Mild Risk 95-97% accuracy > longer expiry,META,0,735.71,sell 675.0 - buy 665.0,665.0,675.0,134.0,747.72,2025-08-20T20:38:08.621961,3,2025-09-12,August,2025-08-20T13:51:13.177308,order placed,735.14,None,None,134.0,2025-08-20T20:37:55.192158
4,547,2025-08-20T04:41:11.777778,Bull Put,Mild Risk 95-97% accuracy > longer expiry,GOOGL,0,194.9,sell 185.0 - buy 175.0,175.0,185.0,138.0,199.32,2025-08-20T20:38:08.568053,2,2025-09-12,August,None,None,NaN,None,None,NaN,None
5,546,2025-08-20T04:41:11.694723,Bull Put,Mild Risk 95-97% accuracy > longer expiry,AAPL,0,225.55,sell 215.0 - buy 205.0,205.0,215.0,127.0,225.90,2025-08-20T20:38:08.595682,1,2025-09-12,August,None,None,NaN,None,None,NaN,None
6,545,2025-08-20T04:40:57.361913,Bull Put,Mild Risk 95-97% accuracy > shorter expiry,SPY,0,632.89,sell 610.0 - buy 600.0,600.0,610.0,77.0,637.94,2025-08-20T20:38:08.739819,3,2025-09-05,August,None,None,NaN,None,None,NaN,None
7,544,2025-08-20T04:40:57.281132,Bull Put,Mild Risk 95-97% accuracy > shorter expiry,MSFT,0,497.47,sell 475.0 - buy 465.0,465.0,475.0,81.0,505.60,2025-08-20T20:38:08.702237,2,2025-09-05,August,None,None,NaN,None,None,NaN,None
8,543,2025-08-20T04:40:57.189854,Bull Put,Mild Risk 95-97% accuracy > shorter expiry,META,0,743.22,sell 685.0 - buy 675.0,675.0,685.0,85.0,747.72,2025-08-20T20:38:08.675062,1,2025-09-05,August,2025-08-20T13:32:30.267268,order placed,742.60,None,None,85.0,2025-08-20T20:37:59.466018
9,542,2025-08-19T04:35:12.638783,Bull Put,Minimal Risk 97-99% accuracy > longer expiry,WMT,0,98.32,sell 94.0 - buy 85.0,85.0,94.0,89.0,100.70,2025-08-19T06:20:51.311940,1,2025-09-12,August,None,None,NaN,None,None,NaN,None


✅ Displayed 16 records with 23 columns
📅 Date range: 2025-08-19T04:34:50.662100 to 2025-08-20T04:41:19.203629


In [5]:
def get_placed_orders():
    """Get orders that have been placed"""
    query = "SELECT id, strategy_type, ticker, trigger_price, estimated_premium, timestamp_of_trigger FROM option_strategies WHERE strategy_status = %s ORDER BY id DESC"
    df = db.execute_query_df(query, ('order placed',))
    print(f"📋 Found {len(df)} placed orders")
    return df

# Show placed orders
try:
    placed_orders = get_placed_orders()
    display(placed_orders)
except Exception as e:
    print(f"❌ Error getting placed orders: {e}")
    print("Trying simpler query...")
    try:
        simple_query = "SELECT COUNT(*) FROM option_strategies WHERE strategy_status = %s"
        count = db.execute_query(simple_query, ('order placed',))[0][0]
        print(f"📊 Found {count} placed orders in database")
    except Exception as e2:
        print(f"❌ Simple count query failed: {e2}")

📋 Found 41 placed orders


,id,strategy_type,ticker,trigger_price,estimated_premium,timestamp_of_trigger
0,481,Bull Put,TSM,233.11,216.00,2025-08-01T13:49:55.858828
1,474,Bull Put,UNH,253.59,135.00,2025-07-31T15:10:31.803712
2,473,Bull Put,NKE,74.1,102.00,2025-08-01T08:34:37.841907
3,472,Bull Put,AAPL,204.28,190.00,2025-08-01T14:41:15.866466
4,471,Bull Put,UNH,257.19,115.00,2025-07-31T13:44:18.361513
5,470,Bear Call,MSFT,559.14,118.00,2025-07-31T05:45:56.872946
6,435,Bull Put,GOOGL,168.81,118.00,2025-06-20T15:01:57.743887
7,431,Bull Put,UNH,303.46,95.00,2025-06-20T15:37:36.049769
8,425,Bull Put,UNH,300.98,126.00,2025-06-19T20:31:01.673866
9,424,Bull Put,TSM,206.95,197.00,2025-06-19T20:27:51.742657


## ⚠️ Delete ALL Rows in Database

**WARNING: This will delete all data in the option_strategies table!**

In [6]:
# Safety cell - prevents accidental execution of dangerous operations
print("🛡️ Safety protection active")

🛡️ Safety protection active


In [7]:
ENABLE_DELETE = False

def delete_all_rows():
    """Delete all rows from PostgreSQL table"""
    if not ENABLE_DELETE:
        print("🛡️ DELETE PROTECTION ENABLED")
        print("Set ENABLE_DELETE = True above to enable deletion")
        return
    
    response = input("⚠️ Delete ALL data? Type 'DELETE ALL': ")
    if response != "DELETE ALL":
        print("❌ Cancelled")
        return
    
    try:
        rows_affected = db.execute_command("DELETE FROM option_strategies")
        remaining = db.execute_query("SELECT COUNT(*) FROM option_strategies")[0][0]
        print(f"🗑️ Deleted {rows_affected} rows, {remaining} remaining")
    except Exception as e:
        print(f"❌ Error: {e}")

delete_all_rows()

🛡️ DELETE PROTECTION ENABLED
Set ENABLE_DELETE = True above to enable deletion


## 🗑️ Drop Table

**WARNING: This will completely remove the option_strategies table!**

In [8]:
def drop_table():
    """Drop the PostgreSQL table"""
    if not ENABLE_DELETE:
        print("🛡️ DROP PROTECTION ENABLED")
        return
    
    response = input("⚠️ DROP ENTIRE TABLE? Type 'DROP TABLE': ")
    if response != "DROP TABLE":
        print("❌ Cancelled")
        return
    
    try:
        db.execute_command("DROP TABLE IF EXISTS option_strategies")
        exists = db.table_exists()
        print(f"🗑️ Table dropped: {not exists}")
    except Exception as e:
        print(f"❌ Error: {e}")

drop_table()

🛡️ DROP PROTECTION ENABLED


## ✏️ Update Single Row

In [9]:
def update_trigger_price(row_id, new_trigger_price):
    """Update trigger price for specific row"""
    try:
        query = "UPDATE option_strategies SET trigger_price = %s WHERE id = %s"
        rows_affected = db.execute_command(query, (new_trigger_price, row_id))
        
        if rows_affected > 0:
            print(f"✅ Updated trigger price to {new_trigger_price} for ID {row_id}")
        else:
            print(f"⚠️ ID {row_id} not found")
    except Exception as e:
        print(f"❌ Error: {e}")

# Example usage
update_trigger_price(7, 55)

✅ Updated trigger price to 55 for ID 7


## 💰 Update Estimated Premium

In [10]:
def update_estimated_premium(row_id, new_premium):
    """Update estimated premium for specific row"""
    try:
        query = "UPDATE option_strategies SET estimated_premium = %s WHERE id = %s"
        rows_affected = db.execute_command(query, (new_premium, row_id))
        
        if rows_affected > 0:
            print(f"✅ Updated premium to {new_premium} for ID {row_id}")
        else:
            print(f"⚠️ ID {row_id} not found")
    except Exception as e:
        print(f"❌ Error: {e}")

# Example usage
update_estimated_premium(496, 10)

✅ Updated premium to 10 for ID 496


## 📈 Update Trigger Price

In [11]:
# Alternative example with different ID
update_trigger_price(496, 100)

✅ Updated trigger price to 100 for ID 496


## 🏷️ Update Ticker Symbol

In [12]:
def update_ticker_symbol(row_id, new_ticker):
    """Update ticker symbol for specific row"""
    try:
        query = "UPDATE option_strategies SET ticker = %s WHERE id = %s"
        rows_affected = db.execute_command(query, (new_ticker, row_id))
        
        if rows_affected > 0:
            print(f"✅ Updated ticker to '{new_ticker}' for ID {row_id}")
        else:
            print(f"⚠️ ID {row_id} not found")
    except Exception as e:
        print(f"❌ Error: {e}")

# Example usage
update_ticker_symbol(192, 'JNJ')

✅ Updated ticker to 'JNJ' for ID 192


In [13]:
def update_strike_buy(row_id, new_strike_buy):
    """Update strike_buy price for specific row"""
    try:
        query = "UPDATE option_strategies SET strike_buy = %s WHERE id = %s"
        rows_affected = db.execute_command(query, (new_strike_buy, row_id))
        
        if rows_affected > 0:
            print(f"✅ Updated strike_buy to {new_strike_buy} for ID {row_id}")
        else:
            print(f"⚠️ ID {row_id} not found")
    except Exception as e:
        print(f"❌ Error: {e}")

# Example usage
update_strike_buy(419, 583.0)

✅ Updated strike_buy to 583.0 for ID 419


## 📊 Bulk Update Strategy Status

In [24]:
def update_strategy_status_bulk(new_status='None', old_status='order placed', target_date='2025-08-20'):
    """Update strategy status in bulk for specific date"""
    try:
        query = """UPDATE option_strategies SET strategy_status = %s 
                   WHERE strategy_status = %s AND DATE(scrape_date) = %s"""
        rows_affected = db.execute_command(query, (new_status, old_status, target_date))
        print(f"✅ Updated {rows_affected} rows: '{old_status}' → '{new_status}' for {target_date}")
    except Exception as e:
        print(f"❌ Error: {e}")

# Example usage
update_strategy_status_bulk()

✅ Updated 4 rows: 'order placed' → 'None' for 2025-08-20


## 📅 Bulk Update Scrape Date

In [15]:
def update_scrape_date_bulk(new_date='2025-08-01', old_date='2025-07-31'):
    """Update scrape dates in bulk"""
    try:
        query = "UPDATE option_strategies SET scrape_date = %s WHERE DATE(scrape_date) = %s"
        rows_affected = db.execute_command(query, (new_date, old_date))
        print(f"✅ Updated {rows_affected} rows: {old_date} → {new_date}")
    except Exception as e:
        print(f"❌ Error: {e}")

# Example usage
update_scrape_date_bulk()

✅ Updated 0 rows: 2025-07-31 → 2025-08-01


## ➕ Add New Column

In [16]:
def add_column_to_table(column_name, column_type='TEXT'):
    """Add a new column to the PostgreSQL table"""
    try:
        # Check if column exists first
        table_info = db.get_table_info()
        existing_columns = [col[0] for col in table_info]
        
        if column_name in existing_columns:
            print(f"⚠️ Column '{column_name}' already exists")
            return
        
        query = f"ALTER TABLE option_strategies ADD COLUMN {column_name} {column_type}"
        db.execute_command(query)
        print(f"✅ Added column '{column_name}' ({column_type})")
    except Exception as e:
        print(f"❌ Error: {e}")

# Example usage
add_column_to_table('timestamp_of_order', 'TIMESTAMP')

⚠️ Column 'timestamp_of_order' already exists


## 🔍 Query Section of Database

In [17]:
def query_by_date_and_status(start_date='2025-03-29', status='triggered'):
    """Query database for specific date and status"""
    try:
        query = """SELECT * FROM option_strategies 
                   WHERE DATE(scrape_date) = %s AND strategy_status = %s"""
        df = db.execute_query_df(query, (start_date, status))
        
        if len(df) == 0:
            print(f"📝 No records found for {start_date} with status '{status}'")
        else:
            print(f"📊 Found {len(df)} records for {start_date} with status '{status}'")
        
        return df
    except Exception as e:
        print(f"❌ Error: {e}")
        return pd.DataFrame()

# Example usage
result = query_by_date_and_status('2025-03-29', 'triggered')
result

📝 No records found for 2025-03-29 with status 'triggered'


,id,scrape_date,strategy_type,tab_name,ticker,er,trigger_price,strike_price,strike_buy,strike_sell,estimated_premium,last_price_when_checked,timestamp_of_price_when_last_checked,item_id,options_expiry_date,date_info,timestamp_of_trigger,strategy_status,price_when_triggered,price_when_order_placed,premium_at_order,premium_when_last_checked,timestamp_of_order


## 🗑️ Delete All Rows Except One

In [18]:
def delete_all_except_one(keep_id=312):
    """Delete all rows except the one with specified ID"""
    if not ENABLE_DELETE:
        print("🛡️ DELETE PROTECTION ENABLED")
        return
    
    response = input(f"⚠️ Delete ALL except ID {keep_id}? Type 'DELETE EXCEPT ONE': ")
    if response != "DELETE EXCEPT ONE":
        print("❌ Cancelled")
        return
    
    try:
        query = "DELETE FROM option_strategies WHERE id != %s"
        rows_affected = db.execute_command(query, (keep_id,))
        print(f"🗑️ Deleted {rows_affected} rows (kept ID {keep_id})")
    except Exception as e:
        print(f"❌ Error: {e}")

# Example usage
delete_all_except_one(312)

🛡️ DELETE PROTECTION ENABLED


## 📅 Delete Rows by Scrape Date

In [19]:
# Enable deletion for this operation
ENABLE_DELETE = True

def delete_rows_by_date(date_to_delete='2025-08-16'):
    """Delete all rows from a specific scrape date"""
    if not ENABLE_DELETE:
        print("🛡️ DELETE PROTECTION ENABLED")
        return
    
    print(f"⚠️ About to delete ALL records from {date_to_delete}")
    print("📊 This will delete approximately 10 records from today's scraper runs")
    print("⌨️  To confirm, type exactly: DELETE DATE")
    response = input("Your confirmation: ")
    if response != "DELETE DATE":
        print(f"❌ Cancelled - you typed: '{response}'")
        print("💡 Remember: type exactly 'DELETE DATE' (all caps, with space)")
        return
    
    try:
        query = "DELETE FROM option_strategies WHERE DATE(scrape_date) = %s"
        rows_affected = db.execute_command(query, (date_to_delete,))
        print(f"🗑️ Successfully deleted {rows_affected} rows from {date_to_delete}")
        
        # Show remaining count
        remaining = db.execute_query("SELECT COUNT(*) FROM option_strategies")[0][0]
        print(f"📊 Remaining records in database: {remaining}")
    except Exception as e:
        print(f"❌ Error: {e}")

# Delete today's records (2025-08-16)
delete_rows_by_date('2025-08-16')

⚠️ About to delete ALL records from 2025-08-16
📊 This will delete approximately 10 records from today's scraper runs
⌨️  To confirm, type exactly: DELETE DATE


KeyboardInterrupt: Interrupted by user

## 🏷️ Change Ticker Symbol (Alternative)

In [ ]:
# Simple ticker update - same functionality as above
update_ticker_symbol(179, 'WMT')

In [ ]:
def update_multiple_fields(row_id, **kwargs):
    """Update multiple fields in a single operation"""
    if not kwargs:
        print("⚠️ No fields to update")
        return
    
    try:
        # Build dynamic query
        set_clauses = []
        params = []
        
        for field, value in kwargs.items():
            set_clauses.append(f"{field} = %s")
            params.append(value)
        
        params.append(row_id)
        
        query = f"UPDATE option_strategies SET {', '.join(set_clauses)} WHERE id = %s"
        rows_affected = db.execute_command(query, tuple(params))
        
        if rows_affected > 0:
            updates = [f"{k}={v}" for k, v in kwargs.items()]
            print(f"✅ Updated ID {row_id}: {', '.join(updates)}")
        else:
            print(f"⚠️ ID {row_id} not found")
    except Exception as e:
        print(f"❌ Error: {e}")

# Example usage
update_multiple_fields(312, ticker='BBY', trigger_price=650, strike_buy=680, strike_sell=690)

## 🎯 PostgreSQL Database Utility Functions - Summary

This notebook provides simplified utility functions specifically for PostgreSQL:

### 🔗 **Connection**
- **Automatic**: Secure credentials loaded from `config/credentials.json`
- **PostgreSQL-only**: Optimized for PostgreSQL syntax and performance
- **Error handling**: Clear error messages and graceful failures

### 📊 **Data Operations**
- `get_all_data()` - Get all records ordered by ID
- `get_recent_data(limit)` - Get recent records with valid tickers
- `get_placed_orders()` - Get orders with 'order placed' status

### ✏️ **Update Operations**
- `update_trigger_price(id, price)` - Update trigger price
- `update_estimated_premium(id, premium)` - Update estimated premium
- `update_ticker_symbol(id, ticker)` - Update ticker symbol
- `update_strike_buy(id, price)` - Update strike buy price
- `update_multiple_fields(id, **kwargs)` - Update multiple fields at once

### 📈 **Bulk Operations**
- `update_strategy_status_bulk()` - Bulk status updates by date
- `update_scrape_date_bulk()` - Bulk date updates
- `query_by_date_and_status()` - Query by date and status

### 🛠️ **Schema Operations**
- `add_column_to_table(name, type)` - Add new columns

### 🗑️ **Deletion (Protected)**
- `delete_all_rows()` - Delete all data (requires ENABLE_DELETE = True)
- `delete_all_except_one(id)` - Delete all except specific ID
- `delete_rows_by_date(date)` - Delete by scrape date
- `drop_table()` - Drop entire table

### 🔒 **Safety Features**
- Delete protection (ENABLE_DELETE flag)
- Confirmation prompts for dangerous operations
- PostgreSQL parameter binding (%s) for SQL injection protection
- Comprehensive error handling

**Total lines of code reduced by ~60% while maintaining all functionality**